# This module is for filtering data and add promoter information

In [3]:
import pandas as pd
import scipy.stats
import numpy as np
import statsmodels.stats.multitest as STM
import math
import copy

## 0. Functions

In [14]:
def read_expression_data(input_address):
    df_temp=pd.read_csv(input_address, delimiter=',',usecols=range(1,35))
    count_list=[]
    for x in df_temp['representative_sequence'].to_list():
        if x in df.index:
            count_list.append(df.loc[x,'Total_count'])
        else:
            count_list.append(0)
    df_temp['Total_count_in_linkage_lib']=count_list
    return(df_temp)

In [20]:
def add_sample_specific_relative_expression(input_data_frame):
# This function will take a dataframe and filtere add Sample specific normalized relative expression
    input_data_frame['S1_normalized_relative']=input_data_frame['N_RNA_relative_count_S1']/input_data_frame['N_DNA_relative_count_S1']
    input_data_frame['S2_normalized_relative']=input_data_frame['N_RNA_relative_count_S2']/input_data_frame['N_DNA_relative_count_S2']
    input_data_frame['S3_normalized_relative']=input_data_frame['N_RNA_relative_count_S3']/input_data_frame['N_DNA_relative_count_S3']
    input_data_frame['Mean_normalized_relative']=(input_data_frame['S1_normalized_relative']+input_data_frame['S2_normalized_relative']+input_data_frame['S3_normalized_relative'])/3
    input_data_frame['SD_normalized_relative']=input_data_frame[['S1_normalized_relative','S2_normalized_relative','S3_normalized_relative']].apply(lambda x: ST.stdev(x), axis=1)
    #this i calculate the std for the expression between three samples
    input_data_frame['CV_normalized_relative']=input_data_frame['SD_normalized_relative']/input_data_frame['Mean_normalized_relative']
    return(input_data_frame)

In [23]:
def all_present_total(input_df,cutoff):
    temp_df=input_df[(input_df['DNA_absolute_count_S1']>cutoff)&(input_df['DNA_absolute_count_S2']>cutoff)&(input_df['DNA_absolute_count_S3']>cutoff)]
    return(temp_df)

## 1. Input and Output

### 1.1 Input address

In [4]:
dr1='Data/'
Total_linkage_resolved_address=dr1+'Linkage_barcode_promoter_combined_Summary'#This let me the linkage information between barcode and promoter. No conflict 
YPD_barcode_ref_2_address=dr1+"YPD_new_N6_Q20_barcode_d2_cluster.csv" #This let me know the YPD (new) barcode cluster ID and its central sequence 
output='../Output/'
YPD_expression_address=dr1+'YPD_final_sum_up_data_V2.csv'
YPD_pc_address=dr1+'YPD_final_positive_control_data_V2.csv'
YPD_nc_address=dr1+'YPD_final_negative_control_data_V2.csv'

### 1.2 Output address


In [5]:
df_output_address=dr1+'New_YPD_analysis/Output/'

### 1.3 Read input 

#### 1.3.1 Read linkage information 

In [6]:
#First I try to read the linkage information
df = pd.read_csv(Total_linkage_resolved_address,skiprows=1,header=None, usecols=range(0,6))

In [7]:
df.columns = ['P_Cluster_ID', 'B_Clustser_ID','P_Clustser_seq','B_Cluster_seq','Total_count','Seq_ID']

In [8]:
df=df.set_index('B_Cluster_seq')

#### 1.3.4 Read expression

In [15]:
YPD_expression=read_expression_data(YPD_expression_address)

In [16]:
YPD_expression=YPD_expression.set_index('representative_sequence')

#### 1.3.5 control expression

In [17]:
YPD_expression_pc=read_expression_data(YPD_pc_address)
YPD_expression_nc=read_expression_data(YPD_nc_address)
YPD_expression_pc=YPD_expression_pc.set_index('representative_sequence')
YPD_expression_nc=YPD_expression_nc.set_index('representative_sequence')


#### 1.3.6 add expression based on total count

In [36]:
control_list=[YPD_expression_pc,YPD_expression_nc]

In [37]:
for x in control_list:
    x['expression']=x['RNA_relative_count_Total']/x['DNA_relative_count_Total']
    x['expression_normalized']=x['N_RNA_relative_count_Total']/x['N_DNA_relative_count_Total']
    x=add_sample_specific_relative_expression(x)

In [ ]:
YPD_expression_pc.to_csv('YPD_PC_expression.csv')
YPD_expression_nc.to_csv('YPD_NC_expression.csv')

## 3. Analysis

### 3.1 Expression analysis

#### 3.1.1 filter data: there is DNA count in all three replicate

In [45]:
YPD_new_expression_f=copy.deepcopy(all_present_total(YPD_expression,0))

In [46]:
YPD_new_expression_f.head()

,barcode_cluster.x,DNA_absolute_count_S1,DNA_relative_count_S1,RNA_absolute_count_S1,RNA_relative_count_S1,N_DNA_absolute_count_S1,N_RNA_absolute_count_S1,N_DNA_relative_count_S1,N_RNA_relative_count_S1,DNA_absolute_count_S2,...,N_RNA_relative_count_S3,DNA_absolute_count_Total,DNA_relative_count_Total,RNA_absolute_count_Total,RNA_relative_count_Total,N_DNA_absolute_count_Total,N_RNA_absolute_count_Total,N_DNA_relative_count_Total,N_RNA_relative_count_Total,Total_count_in_linkage_lib
representative_sequence,,,,,,,,,,,,,,,,,,,,,
AAAAAAAAGAGCCCACTGGC,2506774.0,268,200,254,11,0.000006,5.614577e-06,0.000007,0.000010,255,...,0.000004,1083,776,309,38,0.000023,0.000009,0.000026,0.000023,2456
AAAAAAAAGCCTGAATAGAA,2290079.0,139,100,236,5,0.000003,5.216694e-06,0.000004,0.000005,111,...,0.000007,517,376,279,21,0.000011,0.000008,0.000012,0.000013,712
AAAAAAAGTGCTCAGCGAGT,2581991.0,256399,54967,2558235,40470,0.006044,5.654885e-02,0.002034,0.037781,183415,...,0.026730,852125,167355,3460824,143100,0.018164,0.119243,0.005866,0.088626,0
AAAAAACCTGGAGACGCGGT,2297132.0,501,358,3,3,0.000012,6.631390e-08,0.000013,0.000003,287,...,0.000002,1589,1121,18,15,0.000033,0.000001,0.000037,0.000009,1588
AAAAAAGGCAAGCATAGACC,2749051.0,412,287,900,36,0.000010,1.989417e-05,0.000011,0.000034,363,...,0.000024,1748,1227,1294,160,0.000036,0.000047,0.000040,0.000094,5402


In [47]:
YPD_new_expression_f

,barcode_cluster.x,DNA_absolute_count_S1,DNA_relative_count_S1,RNA_absolute_count_S1,RNA_relative_count_S1,N_DNA_absolute_count_S1,N_RNA_absolute_count_S1,N_DNA_relative_count_S1,N_RNA_relative_count_S1,DNA_absolute_count_S2,...,N_RNA_relative_count_S3,DNA_absolute_count_Total,DNA_relative_count_Total,RNA_absolute_count_Total,RNA_relative_count_Total,N_DNA_absolute_count_Total,N_RNA_absolute_count_Total,N_DNA_relative_count_Total,N_RNA_relative_count_Total,Total_count_in_linkage_lib
representative_sequence,,,,,,,,,,,,,,,,,,,,,
AAAAAAAAGAGCCCACTGGC,2506774.0,268,200,254,11,0.000006,5.614577e-06,0.000007,1.026909e-05,255,...,0.000004,1083,776,309,38,0.000023,9.393030e-06,0.000026,0.000023,2456
AAAAAAAAGCCTGAATAGAA,2290079.0,139,100,236,5,0.000003,5.216694e-06,0.000004,4.667767e-06,111,...,0.000007,517,376,279,21,0.000011,8.264679e-06,0.000012,0.000013,712
AAAAAAAGTGCTCAGCGAGT,2581991.0,256399,54967,2558235,40470,0.006044,5.654885e-02,0.002034,3.778091e-02,183415,...,0.026730,852125,167355,3460824,143100,0.018164,1.192430e-01,0.005866,0.088626,0
AAAAAACCTGGAGACGCGGT,2297132.0,501,358,3,3,0.000012,6.631390e-08,0.000013,2.800660e-06,287,...,0.000002,1589,1121,18,15,0.000033,1.097410e-06,0.000037,0.000009,1588
AAAAAAGGCAAGCATAGACC,2749051.0,412,287,900,36,0.000010,1.989417e-05,0.000011,3.360792e-05,363,...,0.000024,1748,1227,1294,160,0.000036,4.706444e-05,0.000040,0.000094,5402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTTTTTAGTAGTAGCGAGA,1745443.0,151,93,0,0,0.000004,0.000000e+00,0.000003,0.000000e+00,57,...,0.000005,320,209,41,12,0.000007,2.908943e-06,0.000007,0.000006,656
TTTTTTTGATTTACAAGAAT,2683802.0,77,63,125,3,0.000002,2.763079e-06,0.000002,2.800660e-06,81,...,0.000007,337,242,179,16,0.000007,6.607598e-06,0.000008,0.000010,265
TTTTTTTGTGCAAGGTGGCG,4728740.0,89,52,0,0,0.000002,0.000000e+00,0.000002,0.000000e+00,46,...,0.000003,187,129,16,6,0.000004,1.135769e-06,0.000005,0.000003,353


In [48]:
df_list=[YPD_new_expression_f]

In [50]:
for x in df_list:
    x['expression']=x['RNA_relative_count_Total']/x['DNA_relative_count_Total']
    x['expression_normalized']=x['N_RNA_relative_count_Total']/x['N_DNA_relative_count_Total']
    x=add_sample_specific_relative_expression(x)

In [31]:
# Add promoter information
YPD_promoter_df = pd.merge(df, YPD_new_expression_f, left_index=True, right_index=True)

In [30]:
# Output
YPD_new_expression_f.to_csv(dr1+'YPD_RD_expression_total.csv', index=True)

In [32]:
YPD_promoter_df.to_csv(dr1+'YPD_promoter_output.csv', index=True)

In [ ]:
# Compare to median expression

In [52]:
def summarize_expression_RD_median(input_df, cutoff_list_DNA,cutoff_list_RNA,percentile_list,input_fdr,
                                        pc_df, nc_df,conversion_factor):
    # I need to specify both DNA and RNA cutoff
    # this version also compare expression to different percentile of positive control
    total_barcode_t,sig_worse_than_pc,sig_worse_than_nc,sig_worse_than_pc_ratio,sig_worse_than_nc_ratio=[],[],[],[],[]
    sig_better_than_pc,sig_better_than_nc,sig_better_than_pc_ratio,sig_better_than_nc_ratio=[],[],[],[]
    
    #list of barcode sequence with significant expression
    
    sig_better_than_pc_list, sig_worse_than_pc_list, sig_better_than_nc_list, sig_worse_than_nc_list = [], [], [], []
    
    # list of barcode under each cutoff
    total_barcode_list = []
    for temp_cut_D in cutoff_list_DNA: # cutoff_list_DNA is the list of cutoff for DNA 
        for temp_cut_R in cutoff_list_RNA:
            # filter the data according to read count for DNA.
            test_df =all_present_total_new(input_df,temp_cut_D,temp_cut_R)
            
            # total number of unique barcode
            temp_total=test_df.shape[0]
            total_barcode_t.append(temp_total)
            
            temp = np.tile(test_df['expression_normalized'].tolist(),(pc_df.shape[0],1)).transpose()*conversion_factor/pc_df['expression_normalized'].tolist()
            temp_pc_fold_matrix = np.array(temp).transpose()
            
            temp = np.tile(test_df['expression_normalized'].tolist(),(nc_df.shape[0],1)).transpose()/nc_df['expression_normalized'].tolist()
            temp_nc_fold_matrix = np.array(temp).transpose()
            #nc control

            #t test 
            temp_t_test = scipy.stats.ttest_1samp(temp_nc_fold_matrix,1)
            temp_nc_statistic = temp_t_test.statistic
            temp_nc_pvalue= temp_t_test.pvalue        

            #FDR correction
            temp_nc_pvalue_adjusted=STM.multipletests(temp_nc_pvalue,alpha=input_fdr,method='fdr_bh')[0]
            temp_sig1 = temp_nc_pvalue_adjusted&(temp_nc_statistic<0) #the number of barcode that has significantly lower expression than negative control
            temp_sig_worse_nc_number=sum(temp_sig1) 
            temp_sig_worse_nc_list = list(test_df[temp_sig1].index)
            temp_sig2 = temp_nc_pvalue_adjusted&(temp_nc_statistic>0) #the number of barcode that has significantly higher expression than negative control
            temp_sig_better_nc_number=sum(temp_sig2)
            temp_sig_better_nc_list = list(test_df[temp_sig2].index)

            for temp_p in percentile_list:
            #pc control
                #using temp_p percentile of PC is equal to fold expression/temp_p
                temp_t_test = scipy.stats.ttest_1samp(temp_pc_fold_matrix/temp_p,1)
                temp_pc_statistic = temp_t_test.statistic
                temp_pc_pvalue= temp_t_test.pvalue



                # FDR correctiojn
                temp_pc_pvalue_adjusted=STM.multipletests(temp_pc_pvalue,alpha=input_fdr,method='fdr_bh')[0]
                temp_sig3 = temp_pc_pvalue_adjusted&(temp_pc_statistic<0)#the number of barcode that has significantly lower expression than positive control
                temp_sig_worse_pc_number=sum(temp_sig3) 
                temp_sig_worse_pc_list = list(test_df[temp_sig3].index)
                temp_sig4 = temp_pc_pvalue_adjusted&(temp_pc_statistic>0) #the number of barcode that has significantly higher expression than postive control
                temp_sig_better_pc_number=sum(temp_sig4)
                temp_sig_better_pc_list = list(test_df[temp_sig4].index)

                sig_worse_than_pc.append(temp_sig_worse_pc_number)
                sig_worse_than_pc_list.append(temp_sig_worse_pc_list)
                sig_worse_than_nc.append(temp_sig_worse_nc_number)
                sig_worse_than_nc_list.append(temp_sig_worse_nc_list)
                sig_better_than_pc.append(temp_sig_better_pc_number)
                sig_better_than_pc_list.append(temp_sig_better_pc_list)
                sig_better_than_nc.append(temp_sig_better_nc_number)
                sig_better_than_nc_list.append(temp_sig_better_nc_list)
                sig_worse_than_pc_ratio.append(temp_sig_worse_pc_number/temp_total)
                sig_worse_than_nc_ratio.append(temp_sig_worse_nc_number/temp_total)
                sig_better_than_pc_ratio.append(temp_sig_better_pc_number/temp_total)
                sig_better_than_nc_ratio.append(temp_sig_better_nc_number/temp_total)
                total_barcode_list.append(list(test_df.index))
    temp_df=pd.DataFrame({'DNA_cut_off':np.repeat(cutoff_list_DNA,len(percentile_list)*len(cutoff_list_RNA),axis=0),
                          'RNA_cut_off':list(np.repeat(cutoff_list_RNA,len(percentile_list),axis=0))*len(cutoff_list_DNA),
                          'Total_barcode':np.repeat(total_barcode_t,len(percentile_list),axis=0),
                          'Percentile_of_PC':percentile_list*(len(cutoff_list_DNA)*len(cutoff_list_RNA)),
                          'barcode_list': total_barcode_list,
                          'barcode_better_than_pc':sig_better_than_pc,'better_than_pc_ratio':sig_better_than_pc_ratio,
                          'barcode_better_than_pc_list':sig_better_than_pc_list,
                          'barcode_better_than_nc':sig_better_than_nc,'better_than_nc_ratio':sig_better_than_nc_ratio,
                          'barcode_better_than_nc_list':sig_better_than_nc_list,
                          'barcode_worse_than_pc':sig_worse_than_pc,'worse_than_pc_ratio':sig_worse_than_pc_ratio,
                          'barcode_worse_than_pc_list':sig_worse_than_pc_list,
                          'barcode_worse_than_nc':sig_worse_than_nc,'worse_than_nc_ratio':sig_worse_than_nc_ratio,
                          'barcode_worse_than_nc_list':sig_worse_than_nc_list,})
    return(temp_df)

In [ ]:
# 1.906 is the conversion factor from PSP2 to median expression
YPD_RD_summary_extended= summarize_expression_RD_median(YPD_expression_f,
                                                      [1,5,10,20,50,100,200,400],[0,1,2,5,10,20,50,100],
                                                      [1],0.05,
                                                      YPD_expression_pc,YPD_expression_nc,1.906)

In [ ]:
SCD_RD_summary_extended.to_csv(final_data_address+'SCD_RD_summary_median_extended_220420.csv')